
# Negation Handling in Sentiment Classification

In this activity, we will explore **how negation changes the meaning of text** and how to account for it in sentiment analysis.  
Negation words like *not*, *never*, or *didn't* can completely flip the sentiment of a sentence (for example, *“good”* vs. *“not good”*).

We will experiment with two models to see the effect:
1. A **baseline model** using a standard TF–IDF + Logistic Regression pipeline.  
2. A **negation-aware model** that preprocesses text to merge negation words with the following token (e.g., *not good → not_good*).

By comparing their performance, we’ll understand how simple linguistic preprocessing can improve or alter model behavior.

## Import Libraries

In [5]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split


## Step 1: Use the Provided Dataset

In this activity, we’ll work with a **predefined dataset** of short movie reviews.  
The dataset includes examples with and without **negation**, such as *“The movie was good”* and *“The movie was not good.”*

In [6]:

corpus = [
    # Positive without negation
    ("The movie was good", 1),
    ("This is an excellent film", 1),
    ("The movie was amazing", 1),
    ("The film is wonderful", 1),
    ("I loved every minute of it", 1),
    ("Outstanding performance by all actors", 1),
    ("One of the best movies I've seen", 1),
    ("The cinematography was beautiful", 1),
    ("Perfect soundtrack and visuals", 1),
    ("Highly entertaining and engaging", 1),
    ("The acting was superb", 1),
    ("Great storytelling and pacing", 1),
    ("Absolutely fantastic experience", 1),
    ("I thoroughly enjoyed this film", 1),
    ("Brilliant script and direction", 1),
    ("The plot was captivating", 1),
    ("Wonderful character development", 1),
    ("Top quality production", 1),
    ("I would watch it again", 1),
    ("Exceptional movie experience", 1),

    # Negative without negation
    ("The acting was terrible", 0),
    ("This film was boring", 0),
    ("Waste of time and money", 0),
    ("Poorly written script", 0),
    ("Confusing and hard to follow", 0),
    ("The plot made no sense", 0),
    ("Weak character development", 0),
    ("Disappointing overall experience", 0),
    ("Bad cinematography and editing", 0),
    ("Not worth the ticket price", 0),
    ("The dialogue was awful", 0),
    ("Cheap production quality", 0),
    ("I regret watching this", 0),
    ("Boring and predictable story", 0),
    ("Terrible acting from everyone", 0),
    ("Poorly executed concept", 0),
    ("The worst movie I've seen", 0),
    ("Complete waste of time", 0),
    ("Uninteresting and dull", 0),
    ("I hated every minute", 0),

    # Positive with negation (negation flips to positive)
    ("The plot was not bad", 1),
    ("It was not terrible at all", 1),
    ("I can't believe how good it was", 1),
    ("It wasn't bad", 1),
    ("The movie is not boring", 1),
    ("I couldn't believe how great it was", 1),
    ("It's not a waste of time", 1),
    ("The acting wasn't terrible", 1),
    ("I can't say I didn't enjoy it", 1),
    ("The story isn't confusing", 1),
    ("It wasn't disappointing at all", 1),
    ("I can't find anything wrong with it", 1),
    ("The film is not poorly made", 1),
    ("I wouldn't say it's bad", 1),
    ("It wasn't what I expected in a good way", 1),

    # Negative with negation (negation makes it negative)
    ("The movie was not good", 0),
    ("I didn't like the film", 0),
    ("I do not recommend this movie", 0),
    ("I cannot say I enjoyed it", 0),
    ("The story is not interesting", 0),
    ("I don't think it's worth watching", 0),
    ("The acting wasn't good", 0),
    ("I can't say I enjoyed it", 0),
    ("The plot is not engaging", 0),
    ("I didn't find it entertaining", 0),
    ("The film is not worth watching", 0),
    ("I can't recommend this movie", 0),
    ("The story doesn't make sense", 0),
    ("I wouldn't watch it again", 0),
    ("It's not what I expected in a bad way", 0),
    ("The movie isn't entertaining", 0),
    ("I don't understand the hype", 0),
    ("The acting can't save this film", 0),
    ("I won't recommend this to anyone", 0),
    ("The plot doesn't work at all", 0),
]


# Separate features (X) and labels (y)
X = [t for t, y in corpus]
y = [y for t, y in corpus]


## Step 2.1: Prepare the Data
Write the code to separate the text and labels, and then split them into training and test sets using an 80/20 ratio. Do not forget to stratify based on labels.

In [7]:
import spacy
from spacy import displacy

In [8]:
# Split the dataset into training and testing (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training samples: {len(X_train)}, Test samples: {len(X_test)}")


Training samples: 60, Test samples: 15


## Step 2.2: Your preprocessing Fucntion
Later in this activity, you will also use the following **preprocessing function** inside the negation-aware pipeline.  
This function merges negation words (like *not good → not_good*) so the model can treat them as single meaningful tokens.

In [9]:
def preprocess_negation(text):
    text = re.sub(r'\b(not|n\'t|cannot|can\'t|don\'t|didn\'t|doesn\'t|won\'t|wasn\'t|isn\'t|couldn\'t)\s+(\w+)',
                  r'\1_\2', text, flags=re.IGNORECASE)
    return text

#### Can you understand what does it do?

It detects negation words followed by another word and combines them with an underscore (e.g., “not good” → “not_good”) so the model can understand negation as a single unit.

## Step 3: Define and Train the Models

In this step, you will set up two **Logistic Regression models** for sentiment classification:

1. **Baseline model** – uses standard TF-IDF vectorization without any special handling for negation.  
2. **Negation-aware model** – applies the `preprocess_negation()` function before vectorization to treat phrases like “not good” as a single token.

Both models will be trained on the same data.  
After training, we will later compare their performance to see how much the negation-aware preprocessing improves sentiment detection.

In [10]:
def preprocess_negation(text):
    import re
    return re.sub(
        r"\b(not|n't|cannot|can't|don't|didn't|doesn't|won't|wasn't|isn't|couldn't)\s+(\w+)",
        r"\1_\2",
        text,
        flags=re.IGNORECASE
    )

## Step 4: Evaluate and Compare the Models

Now it’s time to **evaluate both models** on the test set.

Run the following code to:
- Generate predictions from the **baseline** and **negation-aware** models.  
- Compute their **accuracy** and **confusion matrices**.  
- Compare results side by side to see whether the negation-aware preprocessing improves performance.

Pay attention to the printed accuracies and the confusion matrices — they will reveal how well each model handles sentences containing negations.

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Baseline: no negation handling
baseline_clf = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1, 2))),
    ("logreg", LogisticRegression(max_iter=1000))
])
baseline_clf.fit(X_train, y_train)

# Negation-aware: uses the preprocessor
negation_clf = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1, 2), preprocessor=preprocess_negation)),
    ("logreg", LogisticRegression(max_iter=1000))
])
negation_clf.fit(X_train, y_train)

print("Models trained.")


Models trained.


# evaluaton and comparisons

In [12]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Predict
pred_base = baseline_clf.predict(X_test)
pred_neg  = negation_clf.predict(X_test)

# Accuracy
acc_base = accuracy_score(y_test, pred_base)
acc_neg  = accuracy_score(y_test, pred_neg)
print(f"Baseline accuracy:       {acc_base:.3f}")
print(f"Negation-aware accuracy: {acc_neg:.3f}")

# Confusion matrices
print("\nConfusion matrix (baseline):")
print(confusion_matrix(y_test, pred_base))

print("\nConfusion matrix (negation-aware):")
print(confusion_matrix(y_test, pred_neg))

# Quick sanity-check sentences with negation
examples = [
    "not good",
    "not bad",
    "I don't dislike it",
    "I wouldn't recommend it",
    "I can't say it's terrible",
    "This is not amazing",
]
for s in examples:
    b = baseline_clf.predict([s])[0]
    n = negation_clf.predict([s])[0]
    print(f"\n{s}")
    print("  Baseline     →", "Positive" if b==1 else "Negative")
    print("  Negation-aware →", "Positive" if n==1 else "Negative")


Baseline accuracy:       0.333
Negation-aware accuracy: 0.333

Confusion matrix (baseline):
[[5 3]
 [7 0]]

Confusion matrix (negation-aware):
[[5 3]
 [7 0]]

not good
  Baseline     → Positive
  Negation-aware → Negative

not bad
  Baseline     → Positive
  Negation-aware → Positive

I don't dislike it
  Baseline     → Negative
  Negation-aware → Negative

I wouldn't recommend it
  Baseline     → Negative
  Negation-aware → Negative

I can't say it's terrible
  Baseline     → Negative
  Negation-aware → Negative

This is not amazing
  Baseline     → Negative
  Negation-aware → Negative
